In [1]:
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337) # for reproducibility
from keras.utils import np_utils
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)
import cv2
from keras import backend as K
from keras.layers import Activation
from keras.preprocessing.image import img_to_array
import os

Using TensorFlow backend.


In [2]:
im_dir = '/home/choppy/Desktop/CNN/male_female_cnn/examples/'
images = os.listdir(im_dir)
im_names = images[:]
images = list(map(lambda x: os.path.join(im_dir, x), images))
images = np.array(list(map(lambda x: cv2.imread(x), images)), dtype=object)

# im = cv2.imread(im_dir)

In [3]:
model = load_model('final_last8_layers.h5')

In [4]:
images = np.array(list(map(lambda x: cv2.resize(x, (224, 224)), images)))
images = np.array(list(map(lambda x: x.astype("float") / 255.0, images)))
images = np.array(list(map(lambda x: img_to_array(x), images)))
images = np.array(list(map(lambda x: np.expand_dims(x, axis=0), images)))


In [5]:
# (female, male) = model.predict(images)
# esto porque f > m

labels = []
preds = list(map(lambda x: model.predict(x), images))

for p in preds:
    (female, male) = p[0]
    label = "male" if male > female else "female"
    proba = male if male > female else female
    label = "{}: {:.2f}%".format(label, proba * 100)
    labels.append(label)

for i in range(len(preds)):
    print(im_names[i], '\t', labels[i])

kurisu.jpg 	 male: 54.38%
f_china.jpg 	 female: 100.00%
f_sinlentes.jpg 	 female: 98.99%
dvega.png 	 female: 99.93%
f_china02.jpg 	 female: 98.70%
ñom01.jpg 	 male: 60.14%
miguel.png 	 female: 99.67%
dave.jpeg 	 male: 76.57%
m_lentes.jpeg 	 male: 99.97%
m_sinlentes.jpg 	 male: 99.13%
ñom02.jpg 	 male: 52.32%
f_lentes.jpg 	 female: 85.44%


In [4]:
# https://github.com/keras-team/keras/issues/10417

import json
import h5py

def fix_layer0(filename, batch_input_shape, dtype):
    with h5py.File(filename, 'r+') as f:
        model_config = json.loads(f.attrs['model_config'].decode('utf-8'))
        layer0 = model_config['config'][0]['config']
        layer0['batch_input_shape'] = batch_input_shape
        layer0['dtype'] = dtype
        f.attrs['model_config'] = json.dumps(model_config).encode('utf-8')

# Example
fix_layer0('final_last8_layers.h5', [None, 224, 224, 3], 'float32')

# no se como, pero lo hizo funcionar xd
# nota: si descargas el modelo de drive, debe pasar por esta funcion para arreglar el input
# por algun motivo, cagó (es mas, no aparece en el summary, pero igual funciona la cnn)